In [2]:
!pip install tensorflow==2.10.1

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
#importing libraries
import os
import cv2
import itertools
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle
from glob import glob
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import*
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import InceptionV3

ModuleNotFoundError: No module named 'cv2'

In [1]:
#parameters
H, W = 224, 224
channel = 3
IMAGE_SHAPE = [224, 224]
num_class = 3
batch_size = 64
class_names = ["brain_glioma", "brain_menin", "brain_tumor"]

In [ ]:
model_path = "/kaggle/working//models/vgg_for_brain_tumor-v2.h5"
path = "/kaggle/input/multi-cancer/Multi Cancer/Brain Cancer"